In [45]:
#from pandas_datareader import data
from pandas_datareader import data, wb
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import seaborn as sns
from datetime import datetime
sns.set(style='darkgrid', context='talk', palette='Paired')
import warnings
warnings.filterwarnings('ignore')

## Getting Stock Data with Yahoo Finance (Old way)

In [51]:
# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
tickers = ['AAPL', 'MSFT', '^GSPC']

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2010-01-01'
end_date = '2016-12-31'

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
df = yf.download(tickers, start_date, end_date)
df = df.drop('Adj Close', 1)
df = df.drop('Volume', 1)

[*********************100%***********************]  3 of 3 completed


In [52]:
df_stacked = df.stack() 
print(df_stacked)
#df.columns

                        Close         High          Low         Open
Date                                                                
2010-01-04 AAPL     30.572857    30.642857    30.340000    30.490000
           MSFT     30.950001    31.100000    30.590000    30.620001
           ^GSPC  1132.989990  1133.869995  1116.560059  1116.560059
2010-01-05 AAPL     30.625713    30.798571    30.464285    30.657143
           MSFT     30.959999    31.100000    30.639999    30.850000
...                       ...          ...          ...          ...
2016-12-29 MSFT     62.900002    63.200001    62.730000    62.860001
           ^GSPC  2249.260010  2254.510010  2244.560059  2249.500000
2016-12-30 AAPL    115.820000   117.199997   115.430000   116.650002
           MSFT     62.139999    62.990002    62.029999    62.959999
           ^GSPC  2238.830078  2253.580078  2233.620117  2251.610107

[5286 rows x 4 columns]


## Updated (4/14/18): Getting Stock Data with Google Finance
Yahoo finance has changed the structure of its website and as a result the most popular Python packages for retrieving data have stopped functioning properly. Until this is resolved, the following piece of code will provide sufficient data to run the examples in this series of articles. Data is now taken from Google Finance and we are using the ETF "SPY" as proxy for S&P 500 on Google Finance.

Please not that if you use this set of data to run the example, you may be getting slightly different results.

In [ ]:
# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
tickers = ['AAPL', 'MSFT', 'SPY']

# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2010-01-01'
end_date = '2016-12-31'

# Use pandas_reader.data.DataReader to load the desired data. As simple as that.
data = data.DataReader(tickers, 'yahoo', start_date, end_date)

In [ ]:
data.head(9)

In [ ]:
# Getting just the adjusted closing prices. This will return a Pandas DataFrame
# The index in this DataFrame is the major index of the panel_data.
close = data['Close']

# Getting all weekdays between 01/01/2000 and 12/31/2016
all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')

# How do we align the existing prices in adj_close with our new set of dates?
# All we need to do is reindex close using all_weekdays as the new index
close = close.reindex(all_weekdays)

# Reindexing will insert missing values (NaN) for the dates that were not present
# in the original set. To cope with this, we can fill the missing by replacing them
# with the latest available price for each instrument.
close = close.fillna(method='ffill')

close.head(10)

In [ ]:
all_weekdays

In [ ]:
close.describe()

In [ ]:
# Get the MSFT timeseries. This now returns a Pandas Series object indexed by date.
msft = close.loc[:, 'MSFT']

# Calculate the 20 and 100 days moving averages of the closing prices
short_rolling_msft = msft.rolling(window=20).mean()
long_rolling_msft = msft.rolling(window=100).mean()

In [ ]:
# Plot everything by leveraging the very powerful matplotlib package
fig, ax = plt.subplots(figsize=(16,9))

ax.plot(msft.index, msft, label='MSFT')
ax.plot(short_rolling_msft.index, short_rolling_msft, label='20 days rolling')
ax.plot(long_rolling_msft.index, long_rolling_msft, label='100 days rolling')

ax.set_xlabel('Date')
ax.set_ylabel('Adjusted closing price ($)')
ax.legend()
sns.despine()